In [ ]:
%load_ext nb_mypy

# How to Check that a Formula is a Tautology

In this notebook we develop a function <tt>tautology</tt> that takes a formula $f$ from propositional logic and checks whether $f$ is a tautology.  As we represent tautologies as nested tuples, we first have to import the parser for propositional logic.

In [ ]:
%%capture
%run Propositional-Logic-Parser.ipynb

In [ ]:
from typing import TypeVar

In [ ]:
Formula = TypeVar('Formula')
Formula = str | tuple[Formula, ...]

In [ ]:
def parse(s: str) -> Formula:
    parser = LogicParser(s) # type: ignore
    return parser.parse()   # type ignore

`Element` is a generic type variable that describes the type of the elements of a set.

In [ ]:
Element = TypeVar('Element')

As we represent propositional valuations as sets of variables, we need a function to compute all subsets of a given set.  The module <tt>power</tt> provides a function called <tt>allSubsets</tt> such that for a given set $M$ the function call $\texttt{allSubsets}(M)$ computes a list containing all subsets of $M$, that is we have:
$$ \texttt{allSubsets}(M) = \bigl[A \mid A \in 2^M\bigr] $$

In [ ]:
def allSubsets(M: set[Element]) -> list[set[Element]]:
    "Compute a list containing all subsets of the set M"
    if M == set():
        return [ set() ]
    x = M.pop()
    L = allSubsets(M)
    return L + [ K | { x } for K in L ]

In [ ]:
allSubsets({'p', 'q'})

In [ ]:
allSubsets({'p', 'q', 'r'})

To be able to compute all propositional valuations for a given formula $f$ we first need to determine the set of all variables that occur in $f$.  The function $\texttt{collectVars}(f)$ takes a formula $f$ from propositional logic and computes all propositional variables occurring in $f$.  This function is defined recursively.

In [ ]:
def collectVars(f: Formula) -> set[str]:
    "Collect all propositional variables occurring in the formula f."
    match f:
        case p if isinstance(p, str): return { p }
        case ('⊤', ) | ('⊥', ): return set()
        case ('¬', g):  return collectVars(g)
        case (_, g, h): return collectVars(g) | collectVars(h) 
    return None # type: ignore

We have discussed the function <tt>evaluate</tt> previously.  The call 
$\texttt{evaluate}(f, I)$ takes a propsitional formula $f$ and a propositional valuation $I$, where $I$ is represented as a set of propositional variables.  It evaluates $f$ given $I$.

In [ ]:
def evaluate(F: Formula, I: set[str]) -> bool:
    """
    Evaluate the propositional formula f using the propositional valuation I.
    I is represented as a set of variables.
    """
    match F:
        case p if isinstance(p, str): 
            return p in I
        case ('⊤', ):     return True
        case ('⊥', ):     return False
        case ('¬', G):    return not evaluate(G, I)
        case ('∧', G, H): return     evaluate(G, I) and evaluate(H, I)
        case ('∨', G, H): return     evaluate(G, I) or  evaluate(H, I)
        case ('→', G, H): return not evaluate(G, I) or  evaluate(H, I)
        case ('↔', G, H): return     evaluate(G, I) ==  evaluate(H, I)
    return None # type: ignore

Now we are ready to define the function $\texttt{tautology}(f)$ that takes a propositional formula $f$ and checks whether $f$ is a tautology.  If $f$ is a tautology, the function returns <tt>True</tt>, otherwise a set of variables $I$ is returned such that $f$ evaluates to <tt>False</tt> if all variables in $I$ are <tt>True</tt>, while all variables not in $I$ are <tt>False</tt>.

In [ ]:
def tautology(f: Formula) -> bool | set[str]:
    "Check, whether the formula f is a tautology."
    P = collectVars(f)
    for I in allSubsets(P):
        if not evaluate(f, I):
            return I
    return True

The function $\texttt{test}(s)$ takes a string $s$ that can be parsed as a propositional formula and checks whether this formula is a tautology.

In [ ]:
def test(s: str) -> None:
    f = parse(s)
    counterExample = tautology(f);
    if counterExample == True: 
        print('The formula', s, 'is a tautology.')
    else: 
        P = collectVars(f)
        print('The formula ', s, ' is not a tautology.')
        print('Counter example: ')
        for x in P:
            if x in counterExample: # type: ignore
                print(x, "↦ True")
            else:
                print(x, "↦ False")

Let us run a few tests.

The first example is DeMorgan's rule.

In [ ]:
test('¬(p ∨ q) ↔ ¬p ∧ ¬q')

In [ ]:
test('(p → q) → (¬p → q) → q')

In [ ]:
test('(p → q) → (¬p → ¬q)')

In [ ]:
test('(p → q) ↔ (¬q → ¬p)')

In [ ]:
test('¬p ↔ (p → ⊥)')